In [1]:
import os
import nltk
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, Markdown

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# Load gemma tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")

# Load gemma model
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-1.1-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [3]:
def extract_skills(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    skills = [word for word, pos in tagged if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    return skills

def match_skills(user_skills, job_description):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([user_skills, job_description])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

def recommend_job(user_skills):
    matches = [(desc, match_skills(user_skills, desc)) for desc in job_descriptions]
    matches.sort(key=lambda x: x[1], reverse=True)
    return matches[0][0] if matches else None

In [4]:
def generate_resume(user_data, resume_template):
    prompt = f"Create a professional resume for the following user data :\n\n{user_data}\n Resume Template : {resume_template}"
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**input_ids, max_new_tokens=4096)
    resume = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resume

def generate_cover_letter(user_data, job_description):
    prompt = f"Write a cover letter for the following job description:\n\n{job_description}\n\nAnd the following user data:\n\n{user_data}\n"
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**input_ids, max_new_tokens=4096)
    cover_letter = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return cover_letter

In [5]:
## resume template
with open('resume_template.md', 'r') as file:
    resume_template = file.read()

In [6]:
## trending jobs in the market
with open('job_descriptions.json', 'r') as file:
    job_descriptions = json.load(file)

print(job_descriptions[6])

Product Manager experienced in leading cross-functional teams to deliver software products. Strong skills in market research, product lifecycle management, and agile methodologies.


In [7]:
class User:
  def __init__(self):

    ## get user info
    name = input("Enter your name: ")
    mobile = input("Enter your mobile no: ")
    email = input("Enter your email: ")
    experience = input("Enter your experience (years): ")
    skills = input("Enter your skills (comma-separated): ")
    education = input("Enter your education: ")
    passing_year = input("Enter your college passing year: ")
    address = input("Enter your address: ")
    linkedin = input("Enter your LinkedIn profile URL: ")
    bio = input("Enter a brief bio about yourself: ")

    ## save user info
    self.user_data = {
        'name': name,
        'mobile no': mobile,
        'email': email,
        'experience (years)': experience,
        'skills': skills,
        'education': education,
        'passing year': passing_year,
        'address': address,
        'LinkedIn': linkedin,
        'bio': bio
    }

  def get_userData(self):
    ## additional function to extract user data
    return self.user_data

  def get_recommended_job(self):
    user_skills = extract_skills(self.user_data['skills'])
    self.recommended_job = recommend_job(" ".join(user_skills))
    return self.recommended_job

  def get_resume(self, resume_template):
    resume = generate_resume(self.user_data, resume_template)

    display(Markdown(resume))

    # Save the MarkDown content to a .md file
    with open(f"{self.user_data['name']}-resume.md", "w") as f:
      f.write(resume)
      print(f"Resume Saved as : {self.user_data['name']}-resume.md")


  def get_coverletter(self):
    coverletter = generate_cover_letter(self.user_data, self.recommended_job)

    display(Markdown(coverletter))

    # Save the MarkDown content to a .md file
    with open(f"{self.user_data['name']}-cover-letter-{self.recommended_job}.md", "w") as f:
      f.write(coverletter)
      print(f"Cover Letter Saved : {self.user_data['name']}-cover-letter-{self.recommended_job}.md")

In [8]:
user = User()

Enter your name: Samriddha Sil
Enter your mobile no: +91 9830711718
Enter your email: samriddha.sil297@gmail.com
Enter your experience (years): Don't have any experience, I am a fresher
Enter your skills (comma-separated): C++, Java. Python, Machine Learning, Natural Language Processing, Problem Solving, Power BI, Time Management, Communication Skills
Enter your education: B.Tech. in Computer Science and Engineering
Enter your college passing year: 2025
Enter your address: Kp 9, Campus 15, Kiit University
Enter your LinkedIn profile URL: www.linkedin.com/in/sil-samriddha
Enter a brief bio about yourself: Enthusiastic and detail-oriented Computer Science Engineering student from KIIT University, seeking to leverage my technical skills, academic knowledge, and project experience to contribute to the innovative projects at your esteemed organization.


In [9]:
user.get_recommended_job()

'Software Engineer with experience in Python and Machine Learning.'

In [10]:
user.get_resume(resume_template)

Create a professional resume for the following user data :

{'name': 'Samriddha Sil', 'mobile no': '+91 9830711718', 'email': 'samriddha.sil297@gmail.com', 'experience (years)': "Don't have any experience, I am a fresher", 'skills': 'C++, Java. Python, Machine Learning, Natural Language Processing, Problem Solving, Power BI, Time Management, Communication Skills', 'education': 'B.Tech. in Computer Science and Engineering', 'passing year': '2025', 'address': 'Kp 9, Campus 15, Kiit University', 'LinkedIn': 'www.linkedin.com/in/sil-samriddha', 'bio': 'Enthusiastic and detail-oriented Computer Science Engineering student from KIIT University, seeking to leverage my technical skills, academic knowledge, and project experience to contribute to the innovative projects at your esteemed organization.'}
```

**Name:** Samriddha Sil

**Contact Information:**
- Mobile: +91 9830711718
- Email: samriddha.sil297@gmail.com

**Education:**
- B.Tech. in Computer Science and Engineering, KIIT University, 2025

**Experience:**
- Non-existent

**Skills:**
- C++
- Java
- Python
- Machine Learning
- Natural Language Processing
- Problem Solving
- Power BI
- Time Management
- Communication Skills

**Additional Information:**
- Passing year: 2025
- Address: Kp 9, Campus 15, Kiit University

**LinkedIn:** www.linkedin.com/in/sil-samriddha

**Bio:**
Enthusiastic and detail-oriented Computer Science Engineering student from KIIT University, seeking to leverage my technical skills, academic knowledge, and project experience to contribute to the innovative projects at your esteemed organization.

Resume Saved as : Samriddha Sil-resume.md


In [11]:
user.get_coverletter()

Write a cover letter for the following job description:

Software Engineer with experience in Python and Machine Learning.

And the following user data:

{'name': 'Samriddha Sil', 'mobile no': '+91 9830711718', 'email': 'samriddha.sil297@gmail.com', 'experience (years)': "Don't have any experience, I am a fresher", 'skills': 'C++, Java. Python, Machine Learning, Natural Language Processing, Problem Solving, Power BI, Time Management, Communication Skills', 'education': 'B.Tech. in Computer Science and Engineering', 'passing year': '2025', 'address': 'Kp 9, Campus 15, Kiit University', 'LinkedIn': 'www.linkedin.com/in/sil-samriddha', 'bio': 'Enthusiastic and detail-oriented Computer Science Engineering student from KIIT University, seeking to leverage my technical skills, academic knowledge, and project experience to contribute to the innovative projects at your esteemed organization.'}
 
**[Your Name]**
**[Your Address]**
**[City, State, Zip]**
**[Email Address]**
**[Phone Number]**
**[Date]**


**[Company Name]**
**[Company Address]**
**[City, State, Zip]**


**Subject: Application for Software Engineer Position**

Dear [Hiring Manager Name],

I am writing to express my keen interest in the Software Engineer position at [Company Name]. With my strong foundation in Python and Machine Learning, I am confident that I possess the skills and experience necessary to make a significant contribution to your organization.

As a fresher with no prior professional experience, I am eager to learn and grow within a dynamic and innovative environment. My academic background in Computer Science and Engineering has equipped me with a solid foundation in core programming languages, data structures, algorithms, and software design principles.

My passion for Machine Learning has driven me to pursue certifications in TensorFlow and PyTorch. I am proficient in data preprocessing, model training, and evaluation. Additionally, my knowledge of Natural Language Processing and Problem Solving enables me to approach software development with a holistic perspective.

I am proficient in various programming languages and frameworks, including Python, Java, and Power BI. I am also experienced in time management, communication, and project management. My strong analytical and problem-solving skills allow me to identify and resolve complex technical challenges efficiently.

I am eager to join a team of talented professionals who are committed to innovation and excellence. I am confident that my skills and experience align perfectly with the requirements of this position.

I have attached my resume for your review, which provides further details of my qualifications and accomplishments. I am available for an interview at your earliest convenience and would welcome the opportunity to discuss how my expertise can contribute to the success of your organization.

Thank you for your time and consideration.

Sincerely,
[Your Name]

Cover Letter Saved : Samriddha Sil-cover-letter-Software Engineer with experience in Python and Machine Learning..md
